In [1]:
import re
import itertools
import pandas as pd
from tqdm import tqdm
import pickle
import json

In [2]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [4]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [ ]:
dataset=pd.read_csv("../outputFile_updated.csv",sep=",")

In [ ]:
res_names=pd.unique(dataset[['dc.contributor.advisor[]', 'dc.creator.researcher[]']].values.ravel('K'))

In [ ]:
name_index=pd.DataFrame({'r_names':res_names})
name_index['rid']=name_index.index
dataset["advisorId"]=dataset['dc.contributor.advisor[]'].map(name_index.set_index('r_names')['rid'])
dataset["researcherId"]=dataset['dc.creator.researcher[]'].map(name_index.set_index('r_names')['rid'])

In [ ]:
def gen_newIndex(org_dataset, names, similar_n) :  
    for key in similar_n:
        index=names[names['r_names'].isin(similar_n[key])]['rid'].values
        names.loc[names['r_names'].isin(similar_n[key]),'rid']=min(index)
    org_dataset["advId_1"]=org_dataset['dc.contributor.advisor[]'].map(names.set_index('r_names')['rid'])
    org_dataset["studId_1"]=org_dataset['dc.creator.researcher[]'].map(names.set_index('r_names')['rid'])
    print('Done')

In [21]:
sim_names1 = load_obj("0_50000.pkl")

In [22]:
len(sim_names1)

7563

In [23]:
sim_names2 = load_obj("50000_50000.pkl")

In [24]:
len(sim_names2)

4333

In [25]:
sim_names3 = load_obj("100000_25000.pkl")

In [26]:
len(sim_names3)

127

In [27]:
sim_names4 = load_obj('120000_118442.pkl')

In [28]:
len(sim_names4)

780

In [29]:
list(sim_names1.keys())[-5:-1]

[10562849925, 10562881786, 10568125262, 10568145851]

In [ ]:
list(sim_names2.keys())[-5:-1]

In [ ]:
list(sim_names3.keys())[1:5]

In [ ]:
list(sim_names3.keys())[1:5]

In [30]:
tmp =  Merge(sim_names1, sim_names2)

In [31]:
tmp1 = Merge(tmp, sim_names3)

In [32]:
final_sim_names =  Merge(tmp1, sim_names4)

In [33]:
len(final_sim_names)

12803

In [91]:
list(final_sim_names.keys())[-5:]

[31449612003, 31572943986, 31577402863, 31598403411, 31601709736]

In [ ]:
#final_sim_names

In [ ]:
gen_newIndex(dataset, name_index, final_sim_names)

In [ ]:
column_arrangement = ['advisorId', 'researcherId','advId_1','studId_1', 'dc.contributor.advisor[]',
       'dc.creator.researcher[]', 'dc.date.submitted[]', 'dc.language.iso[]',
       'dc.publisher.department[]', 'DepartmentId',
       'dc.publisher.institution[]', 'instituteId', 'dc.title[]', 'thesisId',
       'dc.type.degree', 'dc.date.awarded', 'dc.subject.ddc', 'dc.source.uri',
       'dc.description.abstract', 'dc.title.alternative']

In [ ]:
dataset.head()

In [ ]:
dataset= dataset[column_arrangement]

In [ ]:
#dataset[dataset['dc.contributor.advisor[]'].str.contains('Naushad') & dataset['dc.contributor.advisor[]'].str.contains('Ali')]

In [ ]:
dataset.rename(columns={'dc.contributor.advisor[]': 'advisor_name',"dc.creator.researcher[]":'researcher_name' }, inplace=True)

In [ ]:
dataset.columns

In [ ]:
with open('Disambiguated_names.txt', 'w') as file:
     file.write(json.dumps(final_sim_names))

In [ ]:
dataset.to_csv('Shodhganga_mentorship.csv', sep=',', index = False)

In [ ]:
name_index.to_csv('Shodhganga_indexed.csv', sep = ',', index = False)

In [ ]:
sim_names = load_obj('0_205235.pkl')

In [ ]:
len(sim_names)

In [ ]:
# The second approach by which basline index generated include (name, institution)

In [5]:
sim_names1 = load_obj("result1/0_66037.pkl")

In [6]:
len(sim_names1)

8353

In [7]:
sim_names2 = load_obj("result1/66037_132074.pkl")

In [8]:
len(sim_names2)

3481

In [9]:
sim_names3 = load_obj("result1/132074_198111.pkl")

In [10]:
len(sim_names3)

367

In [11]:
sim_names4 = load_obj("result1/198111_264148.pkl")

In [12]:
len(sim_names4)

587

In [13]:
tmp= Merge(sim_names1, sim_names2)

In [14]:
tmp1= Merge(tmp, sim_names3)

In [15]:
final_dict= Merge(tmp1, sim_names4)

In [16]:
len(final_dict)

12788

In [17]:
#final_dict

In [18]:
def gen_newIndex1(org_dataset, names, similar_n) :  
    for key in similar_n:
        tmp_list=[]
        tmp_list.append(similar_n[key][1])
        tmp_list.append(similar_n[key][3])
        names.loc[names['rid'].isin(tmp_list),'rid']=max(tmp_list)
    org_dataset["advId_1"]=org_dataset['advisor_name_inst'].map(names.set_index('r_names_inst')['rid'])
    org_dataset["studId_1"]=org_dataset['advisee_name_inst'].map(names.set_index('r_names_inst')['rid'])
    print('Done')
    return names, org_dataset

In [19]:
index2=pd.read_csv("basline_index_2.csv")

In [20]:
ment=pd.read_csv("shodhganga_mentorship_w_baseline_index_2.csv")

In [21]:
ment["advisor_name_inst"]=ment["advisor_name"]+"@"+ment["publisher_institution"]
ment["advisee_name_inst"]=ment["researcher_name"]+"@"+ment["publisher_institution"]

In [22]:
ment.head()

,advisorId,researcherId,advId_1,studId_1,advisor_name,researcher_name,date_submitted,language,publisher_dept,DepartmentId,...,tilte,thesisId,degree_type,awarded_date,subject,source_uri,abstract,alternative_title,advisor_name_inst,advisee_name_inst
0,0,84077,0,38335,"Jose, Joseph","Thoppil, John E.",01-01-1998,eng,department of sacred heart college(thevara). d...,D1321,...,Chromosome studies and exploration of chemical...,T121508,phd,16-12-1993,500::Natural sciences & mathematics|580::Plant...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Jose, Joseph@Mahatma Gandhi University","Thoppil, John E.@Mahatma Gandhi University"
1,1,84078,1,75893,"Raju, K. V.","Ancy, V P.",29-05-2015,eng,department of sacred heart college,D2490,...,Fishing industry in Kerala The perils of inter...,T64215,phd,29-03-2016,300::Social sciences,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Raju, K. V.@Mahatma Gandhi University","Ancy, V P.@Mahatma Gandhi University"
2,2,84079,2,75894,"Joseph, K. K.","Sajoy, P B.",24-08-2015,eng,department of sacred heart college,D2490,...,Savings and investment pattern of stock invest...,T134025,phd,29-03-2016,300::Social sciences,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Joseph, K. K.@Mahatma Gandhi University","Sajoy, P B.@Mahatma Gandhi University"
3,3,84080,3,75895,"Thomas, E. M.","Leema, T G.",15-07-2015,eng,department of sacred heart college,D2490,...,An economic analysis of Kole cultivation in Ke...,T78956,phd,02-06-2016,300::Social sciences,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Thomas, E. M.@Mahatma Gandhi University","Leema, T G.@Mahatma Gandhi University"
4,4,84081,4,75896,"Francis, M. S.","Murali, Soumya",28-12-2015,eng,department of sacred heart college,D2490,...,Phytochemical and Molecular Systematics of Ari...,T54905,phd,05-11-2016,300::Social sciences,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Francis, M. S.@Mahatma Gandhi University","Murali, Soumya@Mahatma Gandhi University"


In [23]:
index2.head()

,r_names_inst,rid,r_names,inst
0,"Jose, Joseph@Mahatma Gandhi University",0,"Jose, Joseph",Mahatma Gandhi University
1,"Raju, K. V.@Mahatma Gandhi University",1,"Raju, K. V.",Mahatma Gandhi University
2,"Joseph, K. K.@Mahatma Gandhi University",2,"Joseph, K. K.",Mahatma Gandhi University
3,"Thomas, E. M.@Mahatma Gandhi University",3,"Thomas, E. M.",Mahatma Gandhi University
4,"Francis, M. S.@Mahatma Gandhi University",4,"Francis, M. S.",Mahatma Gandhi University


In [24]:
pd.unique(index2['rid']).shape

(264148,)

In [25]:
dataset, index = gen_newIndex1(ment, index2, final_dict)

Done


In [26]:
ment.columns

Index(['advisorId', 'researcherId', 'advId_1', 'studId_1', 'advisor_name',
       'researcher_name', 'date_submitted', 'language', 'publisher_dept',
       'DepartmentId', 'publisher_institution', 'instituteId', 'tilte',
       'thesisId', 'degree_type', 'awarded_date', 'subject', 'source_uri',
       'abstract', 'alternative_title', 'advisor_name_inst',
       'advisee_name_inst'],
      dtype='object')

In [27]:
pd.unique(index2['rid']).shape

(253363,)

In [ ]:
index2.head(15)

In [ ]:
#ment[ment['advisor_name'].str.contains('Naushad') & ment['advisor_name'].str.contains('Ali')]

In [28]:
ment.to_csv("index2_files/final_mentorship_with_index2_files.csv", index=False)

In [ ]:
ment.columns

In [8]:
sim_dept1 = load_obj("dept_dism/0_2826.pkl")

In [9]:
sim_dept1

{154914: ('department of musicology',
  'department of sociology',
  7.53645578592947,
  7.53645578592947,
  7.53645578592947,
  7.53645578592947),
 354400: ('department of visual art',
  'department of visualart',
  12.0,
  12.0,
  9.380952380952381,
  9.380952380952381),
 529239: ('department of sciol work',
  'department of social work',
  8.631002178649238,
  8.631002178649238,
  8.79838902812587,
  8.79838902812587),
 1363651: ('department of geo physics',
  'department of geophysics',
  12.0,
  12.0,
  9.575,
  9.575),
 1516491: ('dept. of marine geology',
  'department of biology',
  6.287789987789987,
  6.287789987789987,
  7.670191934828158,
  7.670191934828158),
 1516859: ('dept. of marine geology',
  'dept. of marine biology',
  8.452063492063491,
  8.452063492063491,
  8.452063492063491,
  8.452063492063491),
 1727079: ('department of psycology',
  'department of sociology',
  7.9926743626743635,
  7.9926743626743635,
  7.9926743626743635,
  7.9926743626743635),
 2603707: (

<h2>Final Dataset</h2>

In [46]:
sim_names1 = load_obj("index_files4/100000_200000.pkl")

In [93]:
sim_names2 = load_obj("index_files4/200000_281425.pkl")

In [94]:
len(sim_names2)

709

In [11]:
tmp={}
for key in sim_names1:
    if sim_names1[key][2] in tmp and sim_names1[key][0] in tmp.get(sim_names1[key][2]):
        continue
    elif sim_names1[key][0] in tmp and sim_names1[key][2] in tmp.get(sim_names1[key][0]):
        continue
    elif sim_names1[key][2] in tmp and sim_names1[key][0] not in tmp.get(sim_names1[key][2]):
        tmp[sim_names1[key][2]].add(sim_names1[key][0])
    elif sim_names1[key][0] in tmp and sim_names1[key][2] not in tmp.get(sim_names1[key][2]):
        tmp[sim_names1[key][0]].add(sim_names1[key][2])
    else:
        tmp[sim_names1[key][2]]={sim_names1[key][0]}

In [65]:
tmp={}
for key in sim_names1:
    flag=True
    for t in tmp:
        if tmp[t].intersection({sim_names1[key][0],sim_names1[key][2]}):
            flag=False
            tmp[t].add(sim_names1[key][0])
            tmp[t].add(sim_names1[key][2])
    if flag:
        flag=True
        tmp[sim_names1[key][0]]={sim_names1[key][0],sim_names1[key][2]}

In [42]:
tmp={}
for key in sim_names1:
    if sim_names1[key][0] in tmp :
        tmp[sim_names1[key][0]].add(sim_names1[key][2])
    elif sim_names1[key][2] in tmp : 
        tmp[sim_names1[key][2]].add(sim_names1[key][0])
    else:
        tmp[sim_names1[key][0]]={sim_names1[key][2]}

In [99]:
sim_dict={}
for key in sim_names1:
    sim_dict[(sim_names1[key][1],sim_names1[key][3])]=(sim_names1[key][0],sim_names1[key][2])

In [100]:
sim_dict

{(101623, 101616): ('Habeeb Rahiman, P A.', 'Rahiman, P A Habeeb'),
 (101631, 101551): ('Joy Joseph,P', 'Joseph, Joy P.'),
 (101963, 101957): ('Achamma ,Kurian', 'Kurian, Achamma'),
 (101973, 101969): ('Krishnan Nair, P R.', 'Nair, P R Krishnan'),
 (102954, 102937): ('Suma, M N.', 'Suma M N'),
 (103876, 103870): ('Gajwe P M', 'Gajwe, P M.'),
 (104244, 104193): ('Sujathan.P.K', 'Sujathan P K'),
 (104292, 104291): ('venugopalan K', 'Venugopalan K'),
 (104305, 104262): ('Azeez, P. Abdul', 'Abdul Azeez P'),
 (104473, 104436): ('Sudesh, Kumari', 'Kumari, Sudesh'),
 (104691, 104673): ('Mathew, Joseph', 'Joseph, Mathew'),
 (104701, 104700): ('Altaf, Mohd. Musheer.', 'Altaf, Mohd Musheer'),
 (104733, 104702): ('Goel, Nitika', 'Goel, Nitika.'),
 (104849, 104789): ('Irshad Ul Haq, Mohd', 'Haq, Mohd Irshad Ul'),
 (104912, 104832): ('Sultan, Safder', 'Sulthan, Safder'),
 (104954, 104939): ('Khan, Soada Idris.', 'Khan, Soada Idris'),
 (104993, 104965): ('Aziz, Sohel', 'Aziz., Sohel'),
 (105109, 104

In [104]:
sorted(sim_dict.items())

[((101623, 101616), ('Habeeb Rahiman, P A.', 'Rahiman, P A Habeeb')),
 ((101631, 101551), ('Joy Joseph,P', 'Joseph, Joy P.')),
 ((101963, 101957), ('Achamma ,Kurian', 'Kurian, Achamma')),
 ((101973, 101969), ('Krishnan Nair, P R.', 'Nair, P R Krishnan')),
 ((102954, 102937), ('Suma, M N.', 'Suma M N')),
 ((103876, 103870), ('Gajwe P M', 'Gajwe, P M.')),
 ((104244, 104193), ('Sujathan.P.K', 'Sujathan P K')),
 ((104292, 104291), ('venugopalan K', 'Venugopalan K')),
 ((104305, 104262), ('Azeez, P. Abdul', 'Abdul Azeez P')),
 ((104473, 104436), ('Sudesh, Kumari', 'Kumari, Sudesh')),
 ((104691, 104673), ('Mathew, Joseph', 'Joseph, Mathew')),
 ((104701, 104700), ('Altaf, Mohd. Musheer.', 'Altaf, Mohd Musheer')),
 ((104733, 104702), ('Goel, Nitika', 'Goel, Nitika.')),
 ((104849, 104789), ('Irshad Ul Haq, Mohd', 'Haq, Mohd Irshad Ul')),
 ((104912, 104832), ('Sultan, Safder', 'Sulthan, Safder')),
 ((104954, 104939), ('Khan, Soada Idris.', 'Khan, Soada Idris')),
 ((104993, 104965), ('Aziz, Sohel

In [95]:
sim_names2

{41047100115: ('Keshvani, Anita',
  202245,
  'Keshavani, Anita',
  202231,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778),
 41047105540: ('Vijailakshmi, Donka',
  202257,
  'Donka, Vijailakshmi',
  202226,
  12.0,
  12.0,
  9.055555555555555,
  9.055555555555555),
 41047132801: ('Sav, M. L.',
  202313,
  'Sav, M L.',
  202243,
  12.0,
  12.0,
  12.0,
  12.0),
 41047135409: ('Mishra, Manoj Kumar',
  202318,
  'Mishara, Manoj Kumar',
  202266,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778),
 41047206377: ('Kurian, Jomon',
  202441,
  'Kurian,Jomon',
  202414,
  12.0,
  12.0,
  9.345959595959595,
  9.345959595959595),
 41049256626: ('Kuriakose, K. J.',
  203929,
  'Kuriakose K J',
  203804,
  12.0,
  12.0,
  12.0,
  12.0),
 41050807411: ('Deshapande, D V.',
  204567,
  'Deshpande, D V.',
  204556,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778,
  10.62777777777778),
 41054176354: ('Swetha S',
  20560

In [52]:
sim_names1

{20523847356: ('Habeeb Rahiman, P A.',
  101623,
  'Rahiman, P A Habeeb',
  101616,
  12.0,
  12.0,
  9.833333333333334,
  9.833333333333334),
 20523853983: ('Joy Joseph,P',
  101631,
  'Joseph, Joy P.',
  101551,
  12.0,
  12.0,
  9.345959595959595,
  9.345959595959595),
 20524180356: ('Achamma ,Kurian',
  101963,
  'Kurian, Achamma',
  101957,
  12.0,
  12.0,
  9.19047619047619,
  9.19047619047619),
 20524192073: ('Krishnan Nair, P R.',
  101973,
  'Nair, P R Krishnan',
  101969,
  12.0,
  12.0,
  9.477513227513228,
  9.477513227513228),
 20525816612: ('Suma, M N.',
  102954,
  'Suma M N',
  102937,
  12.0,
  12.0,
  12.0,
  12.0),
 20528210910: ('Gajwe P M',
  103876,
  'Gajwe, P M.',
  103870,
  12.0,
  12.0,
  12.0,
  12.0),
 20529408153: ('Sujathan.P.K',
  104244,
  'Sujathan P K',
  104193,
  12.0,
  12.0,
  6.233333333333334,
  6.233333333333334),
 20529574355: ('venugopalan K',
  104292,
  'Venugopalan K',
  104291,
  5.0,
  12.0,
  5.0,
  12.0),
 20529619709: ('Azeez, P. Abdu

In [66]:
tmp

{'Habeeb Rahiman, P A.': {'Habeeb Rahiman, P A.', 'Rahiman, P A Habeeb'},
 'Joy Joseph,P': {'Joseph, Joy P.', 'Joy Joseph,P'},
 'Achamma ,Kurian': {'Achamma ,Kurian', 'Kurian, Achamma'},
 'Krishnan Nair, P R.': {'Krishnan Nair, P R.', 'Nair, P R Krishnan'},
 'Suma, M N.': {'Suma M N', 'Suma, M N.'},
 'Gajwe P M': {'Gajwe P M', 'Gajwe, P M.'},
 'Sujathan.P.K': {'Sujathan P K', 'Sujathan.P.K'},
 'venugopalan K': {'Venugopalan K', 'venugopalan K'},
 'Azeez, P. Abdul': {'Abdul Azeez P', 'Azeez, P. Abdul'},
 'Sudesh, Kumari': {'Kumari, Sudesh', 'Sudesh, Kumari'},
 'Mathew, Joseph': {'Joseph, Mathew', 'Mathew, Joseph'},
 'Altaf, Mohd. Musheer.': {'Altaf, Mohd Musheer', 'Altaf, Mohd. Musheer.'},
 'Goel, Nitika': {'Goel, Nitika', 'Goel, Nitika.'},
 'Irshad Ul Haq, Mohd': {'Haq, Mohd Irshad Ul', 'Irshad Ul Haq, Mohd'},
 'Sultan, Safder': {'Sultan, Safder', 'Sulthan, Safder'},
 'Khan, Soada Idris.': {'Khan, Soada Idris', 'Khan, Soada Idris.'},
 'Aziz, Sohel': {'Aziz, Sohel', 'Aziz., Sohel'},
 'F

In [73]:
cnt=1
for sim in sim_names1:
    count = len([i for i in sim_names1[sim][4:] if i >= 10.37 and i < 10.44 ])
    count1 = len([i for i in sim_names1[sim][4:] if i >10.60 ])
    count2 = len([i for i in sim_names1[sim][4:] if i >10.41 and i <10.44 ])
    if count >1 and count1 == 0:
        print(sim_names1[sim])
        cnt+=1
print(cnt)

('Bayat, Drivish Ali', 123682, 'Bayat, Darvish Ali', 123664, 10.242698412698413, 10.242698412698413, 10.413809523809524, 10.413809523809524)
('Das, Ashok Kumar', 132709, 'Das, Asok Kumar', 132625, 10.128181818181817, 10.128181818181817, 10.423296703296703, 10.423296703296703)
('Singh, Rajesh Kumar', 144552, 'Singh, Rakesh Kumar', 144528, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Bhargav, Amit', 148337, 'Bharghav, Amit', 148312, 10.41151515151515, 10.41151515151515, 10.41151515151515, 10.41151515151515)
('Chaudhari, Sonal L.', 181945, 'Chaudhri, Sonal L.', 181926, 10.43996336996337, 10.43996336996337, 10.43996336996337, 10.43996336996337)
('Singh, Rakesh Kumar', 185375, 'Singh, Rajesh Kumar', 185172, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Yadav, Rakesh Kumar', 186647, 'Yadav, Rajesh Kumar', 186603, 10.435555555555556, 10.435555555555556, 10.598333333333334, 10.598333333333334)
('Singh, Ramesh Kumar', 187205, 'Singh, Rakesh Kumar', 187176, 10.3855555555

In [74]:
cnt=0
for sim in sim_names1:
    count = len([i for i in sim_names1[sim][4:] if i >= 10.37 and i < 10.44 ])
    count1 = len([i for i in sim_names1[sim][4:] if i >10.60 ])
    count2 = len([i for i in sim_names1[sim][4:] if i >10.41 and i <10.44 ])
    if count >1 and count1 == 0 and count2 < 2:
        print(sim_names1[sim])
        cnt+=1
print(cnt)
        

('Singh, Rajesh Kumar', 144552, 'Singh, Rakesh Kumar', 144528, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Rakesh Kumar', 185375, 'Singh, Rajesh Kumar', 185172, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Ramesh Kumar', 187205, 'Singh, Rakesh Kumar', 187176, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Ramesh Kumar', 189950, 'Singh, Rajesh Kumar', 189884, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Rakesh Kumar', 189973, 'Singh, Rajesh Kumar', 189884, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Rakesh Kumar', 189973, 'Singh, Ramesh Kumar', 189950, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Ramesh Kumar', 191801, 'Singh, Rajesh Kumar', 191618, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Tiwari, Ramesh Chand', 191807, 'Tiwari, Rakesh Chand', 191562, 10.243703703703703, 10.243703703703703, 10.386363636363638, 10.386363636363638)
('kumar, E.G.Hemantha'

In [77]:
cnt=0
for sim in sim_names1:
    count = len([i for i in sim_names1[sim][4:] if i >=11 ])
    if count < 1: 
        print(sim_names1[sim])
        cnt+=1
print(cnt)

('Sultan, Safder', 104912, 'Sulthan, Safder', 104832, 10.094444444444445, 10.094444444444445, 10.094444444444445, 10.094444444444445)
('Abdul, Lathif', 105207, 'Latif, Abdul', 105168, 10.189292929292929, 10.189292929292929, 8.058656898656897, 8.058656898656897)
('Tariq, Mohammad Shezan', 105557, 'Tariq, Mohammed Shezan', 105507, 10.134365079365079, 10.134365079365079, 10.30084656084656, 10.30084656084656)
('Ghazala, Praveen', 106958, 'Parveen, Ghazala', 106867, 10.626666666666669, 10.626666666666669, 8.570714285714285, 8.570714285714285)
('Singh, Rajinder', 111733, 'Singh, Harjinder', 111460, 9.511904761904763, 10.086904761904762, 8.279095860566448, 8.811557734204793)
('Dhanalakshmi, R.', 113387, 'Dhanalekshmi, R.', 113262, 9.933809523809526, 9.933809523809526, 10.588571428571429, 10.588571428571429)
('Asok Cheriyan K', 115857, 'Cheriyan, Asok', 115837, 10.62777777777778, 10.62777777777778, 9.216666666666669, 9.216666666666669)
('Sing, Dharmendra', 117169, 'Singh, Dharmendra', 117120, 

In [ ]:
('Singh, Rajinder', 111733, 'Singh, Harjinder')
('Singh, Rajesh Kumar', 144552, 'Singh, Rakesh Kumar', 144528,)
Singh, Amarpal', 156578, 'Singh, Samar pal'
Singh, Rakesh Kumar', 185375, 'Singh, Rajesh Kumar'185172
Singh, Ajit Kumar', 185811, 'Singh, Amit Kumar', 185754
Singh, Vinay Kumar', 185996, 'Singh, Vijay Kumar', 185747,
'Singh, Vijay Kumar', 186453, 'Singh, Vinay Kumar', 186452,
'Yadav, Rakesh Kumar', 186647, 'Yadav, Rajesh Kumar', 186603
Pandey, Ramesh Kumar', 187230, 'Pandey, Rajesh Kumar', 186880,
Pandey, Rakesh Kumar', 187308, 'Pandey, Rajesh Kumar', 186880
Pandey, Rakesh Kumar', 187308, 'Pandey, Ramesh Kumar', 187230,
Singh, Vinay Kumar', 187311, 'Singh, Vijay Kumar', 186825,
'Pandey, Vijay Kumar', 187322, 'Pandey, Vinay Kumar', 186836,
('Singh, Ramesh Kumar', 189950, 'Singh, Rajesh Kumar', 189884, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Rakesh Kumar', 189973, 'Singh, Rajesh Kumar', 189884, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Rakesh Kumar', 189973, 'Singh, Ramesh Kumar', 189950, 10.385555555555555, 10.385555555555555, 10.565, 10.565)
('Singh, Raiesh Kumar', 190056, 'Singh, Rajesh Kumar', 189884, 10.260555555555555, 10.260555555555555, 10.44, 10.44)
('Singh, Raiesh Kumar', 190056, 'Singh, Ramesh Kumar', 189950, 10.260555555555555, 10.260555555555555, 10.44, 10.44)
('Singh, Raiesh Kumar', 190056, 'Singh, Rakesh Kumar', 189973,
 
 ('Tiwari, Ramesh Chand', 191807, 'Tiwari, Rakesh Chand', 191562, 10.243703703703703, 10.243703703703703, 10.386363636363638, 10.386363636363638)
('Yadav, Brajesh Kumar', 191914, 'Yadav, Rajesh Kumar', 191749, 9.571831501831502, 10.537582417582419, 9.608571428571429, 10.462142857142858)

In [78]:
dataset=pd.read_csv("index_files4/final_mod_ment_w_baseline_gen4.csv",sep=",")

In [79]:
dataset.columns

Index(['advisorId', 'researcherId', 'advisor_name', 'researcher_name',
       'dc.date.submitted[]', 'dc.language.iso[]', 'publisher_dept',
       'DepartmentId', 'publisher_institution', 'instituteId', 'dc.title[]',
       'thesisId', 'dc.type.degree', 'dc.date.awarded', 'dc.subject.ddc',
       'dc.source.uri', 'dc.description.abstract', 'dc.title.alternative',
       'advisor_inst_dept', 'stud_inst_dept', 'adv_names_rid',
       'res_names_rid'],
      dtype='object')

In [101]:
dataset[dataset['researcherId']==169382]

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.type.degree,dc.date.awarded,dc.subject.ddc,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid
80504,38943,169382,"Vaijayanti, Shete","Balamani, M.",NaN,eng,faculty of fine arts,D1067,Maharaja Sayajirao University of Baroda,I285,...,phd,NaN,The arts| fine & decorative arts,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Vaijayanti, Shete@I285@D1067","Balamani, M.@I285@D1067","Vaijayanti, Shete@38943","Balamani, M.@169382"


In [102]:
dataset[dataset['advisorId']==169382]

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.type.degree,dc.date.awarded,dc.subject.ddc,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid


In [68]:
for sim in sim_names1:
    count = len([i for i in sim_names1[sim][4:] if i >= 10.20 and i < 10.44 ])
    count1 = len([i for i in sim_names1[sim][4:] if i >10.30 ])
    if count >1 and count1 == 0:
        print(sim_names1[sim])

('Bilawar, R B,', 122620, 'Bilwar,R.B.', 122602, 10.240952380952383, 10.240952380952383, 6.945366300366301, 7.070366300366301)
('Singh, Deepika', 126183, 'Deepak, Singh', 126046, 10.257545787545787, 10.257545787545787, 8.308492063492064, 8.308492063492064)
('Majumdar, Shaili', 182757, 'Majmdar, Shaili', 182735, 10.240952380952383, 10.240952380952383, 10.240952380952383, 10.240952380952383)
('Joseph Iruthayaraj U.', 184235, 'Joseph Irudhayaraj, U.', 184199, 10.01037037037037, 10.01037037037037, 10.233982683982685, 10.233982683982685)
('Singh, Vinay Kumar', 185996, 'Singh, Vijay Kumar', 185747, 10.08436507936508, 10.08436507936508, 10.267513227513227, 10.267513227513227)
('Singh, Vijay Kumar', 186453, 'Singh, Vinay Kumar', 186452, 10.08436507936508, 10.08436507936508, 10.267513227513227, 10.267513227513227)
('Singh, Vinay Kumar', 187311, 'Singh, Vijay Kumar', 186825, 10.08436507936508, 10.08436507936508, 10.267513227513227, 10.267513227513227)
('Pandey, Vijay Kumar', 187322, 'Pandey, Vin